In [3]:
import pandas as pd
import numpy as np
import random
from random import choice, sample, choices
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

# Loading Data

In [4]:
user_subset = pd.read_excel("../data/finerydata.xlsx", sheet_name = "7. user_subset")
item_subset = pd.read_excel("../data/finerydata.xlsx", sheet_name = "8. item_subset")
wishlist = pd.read_excel("../data/finerydata.xlsx", sheet_name = "12. wishlist_items")
influencers = pd.read_excel("../data/finerydata.xlsx", sheet_name = "13. user_influencer")
users_set_100 = pd.read_excel("../data/finerydata.xlsx", sheet_name = "14. 100_users_set")
item_set_100 = pd.read_excel("../data/finerydata.xlsx", sheet_name = "15. 100_users_item_set")
influencers_100 = pd.read_excel("../data/finerydata.xlsx", sheet_name = "16. 100_user_influencer")
category = pd.read_excel("../data/finerydata.xlsx", sheet_name = "9. category_ids")
brand_affinity = pd.read_excel("../data/finerydata.xlsx", sheet_name = "10. brands_affinity")
reference_color = pd.read_excel("../data/finerydata.xlsx", sheet_name = "2. reference_color")
occasion = pd.read_excel("../data/finerydata.xlsx", sheet_name = "4. sample_occasion")
color_rule = pd.read_excel("../data/finerydata.xlsx", sheet_name = "5. influencer_color_rules", header = None)
cat_subcat = pd.read_excel("../data/finerydata.xlsx", sheet_name = "6. category_and_subcategory", header = None)

# Combining the Data

In [5]:
print("User Subset: " + str(user_subset.shape))
print("Item Subset: " + str(item_subset.shape))
print("Wishlist: " + str(wishlist.shape))
print("Influencers: " + str(influencers.shape))
print("100 User Subset: " + str(users_set_100.shape))
print("100 Item Subset: " + str(item_set_100.shape))
print("100 Influencers: " + str(influencers_100.shape))
print("Category ID: " + str(category.shape))
print("Brand Affinity: " + str(brand_affinity.shape))
print("Reference Color: " + str(reference_color.shape))
print("Occasion: " + str(occasion.shape))
print("Influencer Color Rules: " + str(color_rule.shape))
print("Category and Subcategory: " + str(cat_subcat.shape))

User Subset: (20, 96)
Item Subset: (6221, 62)
Wishlist: (164, 31)
Influencers: (15, 2)
100 User Subset: (100, 96)
100 Item Subset: (8371, 62)
100 Influencers: (73, 2)
Category ID: (78, 2)
Brand Affinity: (31, 7)
Reference Color: (657, 7)
Occasion: (12, 1)
Influencer Color Rules: (81, 23)
Category and Subcategory: (56, 10)


In [6]:
brand_affinity1 = brand_affinity.iloc[1:,:3]
brand_affinity1.columns = ['brand_1', 'brand_2', 'score']
brand_affinity2 = brand_affinity.iloc[1:,4:]
brand_affinity2.columns = ['brand_1', 'brand_2', 'score']
brand_aff = brand_affinity1.append(brand_affinity2, ignore_index=True)
brand_aff.shape

(60, 3)

In [7]:
all_user = user_subset.append(users_set_100, ignore_index=True, sort=False)
all_user.shape

(120, 96)

In [8]:
all_influencer = influencers.append(influencers_100, ignore_index=True, sort=False)
all_influencer.shape

(88, 2)

In [9]:
all_item = item_subset.append(item_set_100, ignore_index=True, sort=False)
all_item['wishlist'] = 0
wishlist['wishlist'] = 1
all_item.shape

(14592, 63)

In [10]:
all_item['product_id'] = all_item['product_id'].fillna(all_item['item_id'])

In [11]:
wishlist.rename(columns={'userid': 'user_id', 
                         'itemid': 'product_id', 
                         'itemname': 'item_name',
                         'brandname_lower': 'brand_id',
                         'store_lower': 'store_id',
                         'itemname_lower': 'item_name_lower', 
                         'itemcategory': 'product_category_id',
                         'deleted': 'is_deleted',
                         'price': 'paid_price', 
                         'origprice': 'list_price'
                        }, inplace=True)

In [12]:
item_wishlist = all_item.append(wishlist, ignore_index=True, sort=False)
item_wishlist.shape

(14756, 81)

In [13]:
category = category.dropna(how='all') 
category.head(10)

,Category Name,Category ID
0,Tops,110.0
1,Blouses,111.0
2,T Shirts,112.0
3,Tanks,113.0
4,Knits,114.0
6,Bottoms,120.0
7,Skirts,121.0
8,Shorts,122.0
9,Pants,123.0
10,Jeans,124.0


In [14]:
cat_subcat = cat_subcat.iloc[:,:1].dropna(how='all')
cat_subcat_ = cat_subcat[0].str.split(": ", n = 1, expand = True)
cat_subcat_['category_name'] = cat_subcat_[0]
cat_subcat_['occasion'] = cat_subcat_[1]
cat_subcat_ = cat_subcat_[['category_name', 'occasion']]
all_subcat = 'Beach, Work, School, Errands, Dinner, Concert, Night Life, Cocktail, Brunch, Interview, Movies, Formal'
cat_subcat_['occasion'].replace([None], all_subcat, inplace=True)
cat_subcat_.head(10)

,category_name,occasion
0,Shoes,"Beach, Work, School, Errands, Dinner, Concert,..."
1,Heels,"Work, Formal, Night Life, Cocktail, Interview,..."
2,Sneakers,"School, Errands, Concert, Movies, Workout, Din..."
3,Sandals,"Beach, School, Errands, Brunch, Movies, Dinner..."
4,Boots,"Work, School, Errands, Dinner, Concert, Movies..."
5,Flats,"Beach, Work, School, Errands, Dinner, Concert,..."
7,Tops,"Beach, Work, School, Errands, Dinner, Concert,..."
8,Blouses,"Work, School, Dinner, Concert, Night Life, Bru..."
9,T-shirts,"Beach, Work, School, Errands, Dinner, Concert,..."
10,Tanks,"Beach, School, Errands, Concert, Night Life, B..."


In [15]:
category = category.replace({'Category Name' : {'T Shirts' : 'T-shirts', 'Knits' : 'Sweaters', 
                                     'Jeans' : 'Denim', 'Jumpsuits': 'Jumpsuit',
                                    'Hoodies': 'Sweatshirt', 'Boots & Booties': 'Boots'}})
nrows = cat_subcat_.shape[0]
category_sub = category.merge(cat_subcat_[:nrows-11], how = "left", left_on = "Category Name", right_on = 'category_name')
category_sub.head()

,Category Name,Category ID,category_name,occasion
0,Tops,110.0,Tops,"Beach, Work, School, Errands, Dinner, Concert,..."
1,Blouses,111.0,Blouses,"Work, School, Dinner, Concert, Night Life, Bru..."
2,T-shirts,112.0,T-shirts,"Beach, Work, School, Errands, Dinner, Concert,..."
3,Tanks,113.0,Tanks,"Beach, School, Errands, Concert, Night Life, B..."
4,Sweaters,114.0,Sweaters,"Work, School, Errands, Dinner, Concert, Brunch..."


In [16]:
item_wishlist = item_wishlist.merge(category_sub.drop("category_name", axis = 1), how = "left", left_on = "product_category_id", right_on = 'Category ID')
item_wishlist.shape

(14756, 84)

# User Data

In [17]:
all_user.head()

,user_id,first_name,last_name,address_zip,gender,signup_date,date_last_login,visited_wishlist,visited_shopping,finished_approval,...,context_device_model,context_device_type,context_network_wifi,device_advertising_id_blocked,context_network_cellular,context_network_carrier,context_os_version,locale_us,context_locale,style_push_permission
0,SBmFVmiyziheGcUXF1506291233259,NYCDSA,NYCDSA,NaN,female,2017-09-24 17:13:53,2017-09-24 17:13:54,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,DHvdjwEmqSXpYONGd1514587641548,NYCDSA,NYCDSA,NaN,female,2017-12-29 17:47:21,2017-12-29 18:38:27,True,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,rboNfQfmMjQrAzLAX1531423951169,NYCDSA,NYCDSA,NaN,other,2018-07-12 14:32:31,2018-07-12 14:32:31,False,False,False,...,"iPhone9,2",iPhone9,1.0,1.0,0.0,Verizon,11.4,1.0,en-US,True
3,RZsOYGrUlkXWxYGSi1532103589490,NYCDSA,NYCDSA,NaN,other,2018-07-20 11:19:49,2018-07-20 11:19:49,False,False,False,...,"iPhone10,5",iPhone10,1.0,0.0,0.0,AT&T,11.4.1,1.0,en-US,True
4,ZsFZyqyShAPBMwkLu1534274912488,NYCDSA,NYCDSA,NaN,female,2018-08-14 14:28:32,2018-08-14 14:28:32,False,False,False,...,"iPhone8,1",iPhone8,1.0,0.0,0.0,AT&T,11.3,1.0,en-US,False


In [18]:
all_user.columns

Index(['user_id', 'first_name', 'last_name', 'address_zip', 'gender',
       'signup_date', 'date_last_login', 'visited_wishlist',
       'visited_shopping', 'finished_approval', 'has_shared', 'safari_ext',
       'chrome_ext', 'firefox_ext', 'registered', 'push_enabled',
       'subscribed_onboarding_emails', 'subscribed_onsale_alerts',
       'subscribed_return_reminders', 'subscribed_finery_updates',
       'subscribed_misc_emails', 'subscribed_no_emails',
       'wishlist_notifications', 'timezone', 'set_avatar', 'style_age_range',
       'style_age_range_group', 'items_in_wardrobe', 'items_in_wishlist',
       'location_latitude', 'location_longitude', 'notifications_enabled_ios',
       'style_brands_selected', 'style_size_preference_none',
       'style_size_preference_petite', 'style_size_preference_extra_long',
       'style_size_preference_plus', 'style_size_preference_maternity',
       'style_size_preference_skipped', 'email_provided',
       'notifications_enabled_desktop'

In [19]:
col_names = ['user_id', 'style_age_range_group', 'style_vibe', 
            'style_most_important_active', 'style_most_important_any',
            'style_most_important_beach', 'style_most_important_dress',
            'style_most_important_bags', 'style_most_important_jeans',
            'style_most_important_jump', 'style_most_important_nothing',
            'style_most_important_outwear', 'style_most_important_pants',
            'style_most_important_shoes', 'style_most_important_tops',
            'style_shopping_pref_gaps',
            'style_shopping_pref_other', 'style_shopping_pref_trips',
            'style_shopping_pref_events', 'style_shopping_pref_work',
            'style_shopping_pref_wish', 'style_shopping_pref_organize',
            'style_shopping_pref_inspo']
subset_user = all_user[col_names]
subset_user.sample(10)

,user_id,style_age_range_group,style_vibe,style_most_important_active,style_most_important_any,style_most_important_beach,style_most_important_dress,style_most_important_bags,style_most_important_jeans,style_most_important_jump,...,style_most_important_shoes,style_most_important_tops,style_shopping_pref_gaps,style_shopping_pref_other,style_shopping_pref_trips,style_shopping_pref_events,style_shopping_pref_work,style_shopping_pref_wish,style_shopping_pref_organize,style_shopping_pref_inspo
45,NtsBKpbNClDDpOfxo1533737578164,1.0,"Durham, NC 27713, USA",0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
25,lTbOURAMnUHxzFLDe1524543123801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,zQLmxXWxdpWlvWoCU1536792275462,3.0,Aliso Viejo,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
91,nFwMokjyOpgInoLMG1529282181454,2.0,Orlando,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
106,KJhFMbviATDhfMRVM1541784526092,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,YudzYKvhlJOftwiQS1533990431896,4.0,Haverhill,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
65,VzynjnjuTvNovmpoh1548724515964,0.0,Marietta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
54,lhJVOyplPaiNkHVEV1539890834190,0.0,Hebron,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
87,ZcXluwdEXFlwqAXQK1527387850533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,yZIcDJCRfeunHavTa1541215258673,1.0,Miami Springs,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [20]:
subset_user['style_vibe'] = subset_user['style_vibe'].str.split(",").str[0].str.lower()
subset_user["style_age_range_group"] = subset_user["style_age_range_group"].apply(lambda x: choices([0, 1, 2], [0.4, 0.3, 0.3])[0] if np.isnan(x) else x)
subset_user["style_vibe"] = subset_user["style_vibe"].fillna(random.choices(subset_user['style_vibe'].unique())[0])

col_names = subset_user.filter(like='style_shopping').columns
col_names = col_names.append(subset_user.filter(like='style_most_important').columns)
subset_user[col_names] = subset_user[col_names].fillna(choices([0,1])[0])

In [21]:
temp = subset_user.isnull().sum().sort_values(ascending = False)
temp[temp != 0]

Series([], dtype: int64)

In [22]:
subset_user.sample(10)

,user_id,style_age_range_group,style_vibe,style_most_important_active,style_most_important_any,style_most_important_beach,style_most_important_dress,style_most_important_bags,style_most_important_jeans,style_most_important_jump,...,style_most_important_shoes,style_most_important_tops,style_shopping_pref_gaps,style_shopping_pref_other,style_shopping_pref_trips,style_shopping_pref_events,style_shopping_pref_work,style_shopping_pref_wish,style_shopping_pref_organize,style_shopping_pref_inspo
0,SBmFVmiyziheGcUXF1506291233259,1.0,hebron,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,oEbkVbMoHtZsjxFlQ1531437626855,0.0,hebron,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
37,YudzYKvhlJOftwiQS1533990431896,4.0,haverhill,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
40,EepjfcrrjDCVkSKLV1543110769237,0.0,huntington beach,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
66,mmWSBdAUHcNggjzBg1530478100506,1.0,west jordan,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
31,qhgEwGZvsLjvCAiVE1534459103461,1.0,atlanta,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
97,EqntCbpopoKSFtPnw1528762526242,1.0,dover,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,KMHJBRMJIkrCcUcrA1539389830936,2.0,village,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
64,GaCXNOFTkNMIWqCSi1534871033178,1.0,chicago,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
106,KJhFMbviATDhfMRVM1541784526092,0.0,hebron,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Item Data

In [23]:
item_wishlist.head()

,_id,item_id,user_id,brand_id,user_provided_brand_name,parsed_brand_name,store_id,user_provided_store_name,parsed_store_name,product_id,...,sale,saleprice,saleurl,share_item_id,saledate,collections,stylesdate,Category Name,Category ID,occasion
0,209561,58d5c57b6b21659a9bf9afc7,ekteZyevBdOMLFZds1490404459978,loft,NaN,Loft,loft,NaN,loft,58d5239899563e3bf98e7033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jackets,152.0,"Work, School, Errands, Dinner, Formal, Concert..."
1,209593,58d5c59f6b21659a9bf9affc,ekteZyevBdOMLFZds1490404459978,NaN,NaN,NaN,nordstrom,NaN,Nordstrom,58d5c59f7c8a96c9a044f19d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miscellaneous,600.0,NaN
2,209657,58d5c54d6b21659a9bf9afa2,ekteZyevBdOMLFZds1490404459978,NaN,NaN,NaN,nordstrom,NaN,Nordstrom,58d5c54d5e4fefcabb201c3c,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miscellaneous,600.0,NaN
3,209881,58d5c6bc6b21659a9bf9b102,ekteZyevBdOMLFZds1490404459978,NaN,NaN,NaN,sephora,NaN,Sephora,58d56e9ec9f4693c00b913e1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Beauty,500.0,NaN
4,209913,58d5c7736b21659a9bf9b144,ekteZyevBdOMLFZds1490404459978,fabletics,NaN,Fabletics,fabletics,NaN,Fabletics,58d13a07b55a7764b1a607d4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Activewear,160.0,NaN


In [24]:
item_wishlist.columns

Index(['_id', 'item_id', 'user_id', 'brand_id', 'user_provided_brand_name',
       'parsed_brand_name', 'store_id', 'user_provided_store_name',
       'parsed_store_name', 'product_id', 'closet_id', 'is_deleted',
       'delete_reason', 'item_name', 'item_name_lower', 'indix_category',
       'product_category_id', 'order_number', 'currency', 'paid_price',
       'list_price', 'sale_price', 'price_range', 'price_filter',
       'order_total_amt', 'order_discount', 'shipping_total',
       'parsed_sale_price', 'size', 'is_vintage', 'is_favorite', 'email_id',
       'added_ts', 'email_ts', 'email_dt', 'sort_ts', 'approval_status',
       'approval_status_dt', 'delete_ts', 'is_for_sale', 'color_parsed',
       'is_returned', 'returned_ts', 'season_id', 'source', 'is_public',
       'sharelink_short', 'is_visible', 'return_deadline_dt', 'has_duplicates',
       'is_mozenda_item', 'indix_added_ts', 'indix_start_ts', 'donation_descr',
       'donation_dt', 'is_price_adjusted', 'is_preprocess

In [25]:
col_names = ['user_id', 'product_id', 'brand_id', 'parsed_brand_name', 'store_id', 'parsed_store_name', 
             'is_deleted', 'item_name_lower', 'item_name',
             'Category ID', 'Category Name', 'occasion', 'paid_price', 'color_parsed', 'wishlist']
subset_item = item_wishlist[col_names]
subset_item.rename(columns={'Category Name': 'category_name', 'Category ID': 'category_id'}, inplace = True)
subset_item['brand_id'] = subset_item['brand_id'].str.replace(' ', '')
subset_item['brand_id'] = subset_item['brand_id'].str.replace('-', '')
subset_item['brand_id'] = subset_item['brand_id'].str.replace('&', '')
subset_item['store_id'] = subset_item['store_id'].str.replace(' ', '')
subset_item['store_id'] = subset_item['store_id'].str.replace('-', '')
subset_item['store_id'] = subset_item['store_id'].str.replace('&', '')
subset_item.sample(5)

,user_id,product_id,brand_id,parsed_brand_name,store_id,parsed_store_name,is_deleted,item_name_lower,item_name,category_id,category_name,occasion,paid_price,color_parsed,wishlist
12602,YQkgLXYELtgdrweSo1537319606842,SeyobPmqvsnlGvjyw1537320219626,thelimited,The Limited,thelimited,The Limited,False,paisley printed tabard,Paisley Printed Tabard,NaN,NaN,NaN,26.14,purple print,0
3356,mdtvvDgrYGguAGATd1495121490933,uzpkIuUHHkmuGLWQW1538136928186,NaN,NaN,zulily,Zulily,False,lemon legwear gray color block kneehigh socks ...,Lemon Legwear Gray Color Block Kneehigh Socks ...,600.0,Miscellaneous,NaN,7.99,NaN,0
14623,blrLJXDkKZYHNuAVh1528340569558,BQonIAPkVpXoOJczT1528340658895,adidas,NaN,academy,NaN,False,women's cloudfoam qt racer running shoes,Women's cloudfoam QT Racer Running Shoes,NaN,NaN,NaN,59.99,NaN,1
6354,hktGDcSbmApUzRhGi1495117820151,58ce10f6a9b6fb9e60cc67c2,harlowegraham,Harlowe Graham,hautelook,HauteLook,False,3 4 sleeve easy blazer,3 4 Sleeve Easy Blazer,153.0,Blazers,"Work, Dinner, Brunch, Interview",45.00,NaN,0
1790,ekteZyevBdOMLFZds1490404459978,59d3c25de87a33044f65c576,victoriassecret,Victorias Secret,victoriassecret,Victoria's Secret,False,new pima cotton short sl,New Pima Cotton Short Sl,NaN,NaN,NaN,26.00,undefined,0


In [26]:
print(subset_item.shape)
temp = subset_item.isnull().sum().sort_values(ascending = False)
temp[temp != 0]

(14756, 15)


occasion             7939
color_parsed         6898
parsed_brand_name    5597
brand_id             5302
category_name        2106
category_id          2106
parsed_store_name     708
store_id              514
item_name             154
item_name_lower       154
paid_price            141
dtype: int64

# User Number of Item

In [27]:
subset_item["has_product"] = [0 if deleted == True else 1 for deleted in subset_item["is_deleted"]]
user_product = subset_item.groupby(["user_id", "product_id"])[["has_product"]].sum().reset_index()
user_product.sort_values(by = "has_product", ascending = False).head(10)
subset_item.sample(5)

,user_id,product_id,brand_id,parsed_brand_name,store_id,parsed_store_name,is_deleted,item_name_lower,item_name,category_id,category_name,occasion,paid_price,color_parsed,wishlist,has_product
7270,uDlpcEvrISMoLvJKh1517962134688,59d7c845bbad4b001d13e3e7,NaN,NaN,kohls,Kohl's,False,fisher price little people big animal zoo,Fisher Price Little People Big Animal Zoo,NaN,NaN,NaN,52.0,"Color: Multi/None, Size: ONE SIZE, Qty: 1",0,1
11189,nFwMokjyOpgInoLMG1529282181454,TYnKdQmhAJHOAaOmq1529282517122,NaN,NaN,kohls,Kohl's,False,braza bra backless strapless freedom bustier p...,Braza Bra Backless Strapless Freedom Bustier P...,600.0,Miscellaneous,NaN,35.0,"color: beige, size: size 3, qty: 1",0,1
12526,OQzclKDakJmSHAtkZ1533655416477,591d921157be8a1eaadd6bd4,NaN,NaN,sephora,Sephora,False,ghd curve class curl iron,Ghd Curve Class Curl Iron,500.0,Beauty,NaN,0.0,NaN,0,1
11794,OQzclKDakJmSHAtkZ1533655416477,5ab3c8ce473696002a813696,NaN,NaN,nordstrom,Nordstrom,False,mac nude lipstick,Mac Nude Lipstick,510.0,Lips,NaN,17.0,"no color,",0,1
4842,zaUVbGzoQMyxwhxYe1532180138666,5ac4ddbc54c7e5002af30d2e,NaN,NaN,nordstrom,Nordstrom,False,jo malone london lime basil mandarin cologne s...,Jo Malone London Lime Basil Mandarin Cologne S...,530.0,Makeup,NaN,0.0,"no color,",0,1


# User-Influencer Matrix

In [28]:
influencers = ['weworewhat', 'chiaraferragni', 'blaireadiebee', 'somethingnavy', 'hannahbronfman',
               'nicolettemason', 'manrepeller', 'jordynwoods', 'seaofshoes', 'ariellecharnas']

In [29]:
user_influencer = pd.DataFrame(data=0, index=all_user.user_id, columns=influencers)

for row in range(all_influencer.shape[0]):
    user = all_influencer['user_id'][row]
    influ = all_influencer['style_who_inspiries'][row]
    for influencer in influencers:
        if influ.find(influencer) != -1:
            user_influencer[influencer][user] += 1 

user_influencer.head(10)

,weworewhat,chiaraferragni,blaireadiebee,somethingnavy,hannahbronfman,nicolettemason,manrepeller,jordynwoods,seaofshoes,ariellecharnas
user_id,,,,,,,,,,
SBmFVmiyziheGcUXF1506291233259,0,0,0,0,0,0,0,0,0,0
DHvdjwEmqSXpYONGd1514587641548,0,0,0,0,0,0,0,0,0,0
rboNfQfmMjQrAzLAX1531423951169,0,1,1,1,0,0,0,0,0,0
RZsOYGrUlkXWxYGSi1532103589490,0,1,0,0,0,1,0,0,0,0
ZsFZyqyShAPBMwkLu1534274912488,1,1,1,1,0,0,0,0,1,0
wPSJznugsLxGTVzSt1535641042977,1,0,0,0,1,1,0,0,0,0
KMHJBRMJIkrCcUcrA1539389830936,0,1,0,0,0,0,0,0,0,0
zaUVbGzoQMyxwhxYe1532180138666,0,0,0,1,0,0,0,0,0,0
rZGWJDRvvWjBLKfGF1532437479456,1,0,0,0,0,0,0,0,0,0


In [30]:
user_user_byinfluencer = pd.DataFrame(data=cosine_similarity(user_influencer), 
                         index=all_user.user_id, columns=all_user.user_id)
user_user_byinfluencer.head(10)

user_id,SBmFVmiyziheGcUXF1506291233259,DHvdjwEmqSXpYONGd1514587641548,rboNfQfmMjQrAzLAX1531423951169,RZsOYGrUlkXWxYGSi1532103589490,ZsFZyqyShAPBMwkLu1534274912488,wPSJznugsLxGTVzSt1535641042977,KMHJBRMJIkrCcUcrA1539389830936,zaUVbGzoQMyxwhxYe1532180138666,rZGWJDRvvWjBLKfGF1532437479456,bqnoAjxnAbwhntJuU1534200663687,...,kIVuVbYAijKgchjBU1547080163262,IxwWDxsICtdFQonXO1538002122412,yDRpRuRkrutKtmZoH1531428926861,jFJkZZkzfcgdpYaWz1546915104530,bgfAtVdSyWtCCasub1535124717025,riTOVCwwBFvAjaBrX1504480921172,iDDoOGOZnwaAKmzpF1531966340209,aDkFJvBQABdodOhRe1485369350003,IUIwmBWjpuwFTuILv1541366448241,fcxfBirJmkApcCtCb1539277708892
user_id,,,,,,,,,,,,,,,,,,,,,
SBmFVmiyziheGcUXF1506291233259,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
DHvdjwEmqSXpYONGd1514587641548,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
rboNfQfmMjQrAzLAX1531423951169,0.0,0.0,1.000000,0.408248,0.774597,0.000000,0.577350,0.577350,0.000000,0.000000,...,0.000000,0.577350,0.577350,0.00000,0.288675,0.0,0.000000,0.0,0.471405,0.774597
RZsOYGrUlkXWxYGSi1532103589490,0.0,0.0,0.408248,1.000000,0.316228,0.408248,0.707107,0.000000,0.000000,0.000000,...,0.000000,0.707107,0.000000,0.00000,0.707107,0.0,0.000000,0.0,0.577350,0.632456
ZsFZyqyShAPBMwkLu1534274912488,0.0,0.0,0.774597,0.316228,1.000000,0.258199,0.447214,0.447214,0.447214,0.316228,...,0.258199,0.447214,0.447214,0.00000,0.447214,0.0,0.447214,0.0,0.547723,0.600000
wPSJznugsLxGTVzSt1535641042977,0.0,0.0,0.000000,0.408248,0.258199,1.000000,0.000000,0.000000,0.577350,0.408248,...,0.333333,0.000000,0.000000,0.57735,0.288675,0.0,0.577350,0.0,0.707107,0.258199
KMHJBRMJIkrCcUcrA1539389830936,0.0,0.0,0.577350,0.707107,0.447214,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.00000,0.500000,0.0,0.000000,0.0,0.408248,0.447214
zaUVbGzoQMyxwhxYe1532180138666,0.0,0.0,0.577350,0.000000,0.447214,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.447214
rZGWJDRvvWjBLKfGF1532437479456,0.0,0.0,0.000000,0.000000,0.447214,0.577350,0.000000,0.000000,1.000000,0.707107,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,1.000000,0.0,0.408248,0.000000


In [314]:
subset_item.to_csv("../data/subset_item.csv")
user_influencer.to_csv("../data/user_influencer.csv")

# Recommender

Functions

In [105]:
subset_item['category_name'].unique()

array(['Jackets', 'Miscellaneous', 'Beauty', 'Activewear', 'Flats',
       'Kids', 'Blouses', 'Jewelry', nan, 'T-shirts', 'Tanks',
       'Sunglasses', 'Coverups', 'Boots', 'Casual', 'Fragrance', 'Pants',
       'Swimsuits', 'Shorts', 'Clutches', 'Sweatshirt', 'Totes',
       'Bottoms', 'Sandals', 'Sweaters', 'Bras', 'Formal', 'Shoes',
       'Scarves', 'Cocktail', 'Face & Body', 'Short', 'Denim', 'Sneakers',
       'Shoulder', 'Skirts', 'Heels', 'Hair', 'Long', 'Hats', 'Vests',
       'Blazers', 'Tops', 'Leggings', 'Coats', 'Other', 'Lips', 'Makeup',
       'Belts', 'Jumpsuit', 'Wallets', 'Nails', 'Backpacks', 'Dresses',
       'Gloves', 'Top Handle', 'Work', 'Outerwear', 'Accessories',
       'Beachwear', 'Bags'], dtype=object)

In [290]:
def filter_item(list_items = [], list_brand = [], list_occasion = [], list_category = [], list_price = []):
    df = subset_item[~subset_item['category_name'].isin(['Beauty', 'Kids'])]
    if list_items != []:
        df = df[~df['product_id'].isin(list_items)]
    if list_brand != []:
        df = df[df['brand_id'].isin(list_brand)]
    if list_occasion != []:
        list_occasion = "|".join(list_occasion)
        df = df[df['occasion'].str.contains(list_occasion, na = False)]
    if list_category != []:
        df = df[df['category_name'].isin(list_category)]
    if list_price != []:
        if list_price == ['a']:
            df = df[(df['paid_price'] < 25)]
        elif list_price == ['c']:
            df = df[df['paid_price'] >= 70]
        else:
            df = df[(df['paid_price'] >= 25) & (df['paid_price'] < 70)]
    return df

In [240]:
def fill_table(df, product_id, n_items):
    recommendation = pd.DataFrame(data = 0, index = range(n_items), 
                                  columns = ['Item Name', 'Brand', 'Store', 
                                             'Category', 'Price', 'Color', 'Occasion'])
    for row, item in enumerate (product_id):
        try:
            recommendation['Item Name'][row] = df[df['product_id'] == item]['item_name'].mode()[0]
        except IndexError:
            recommendation['Item Name'][row] = 'No Item Name'
        try:
            recommendation['Brand'][row] = df[df['product_id'] == item]['parsed_brand_name'].mode()[0]
        except IndexError:
            recommendation['Brand'][row] = 'No Brand'
        try:
            recommendation['Store'][row] = df[df['product_id'] == item]['parsed_store_name'].mode()[0]
        except IndexError:
            recommendation['Store'][row] = 'No Store'
        try:
            recommendation['Category'][row] = df[df['product_id'] == item]['category_name'].mode()[0]
        except IndexError:
            recommendation['Category'][row] = 'No Category'
        try:
            recommendation['Price'][row] = df[df['product_id'] == item]['paid_price'].mean()
        except IndexError:
            recommendation['Price'][row] = 'No Price'
        try:
            recommendation['Color'][row] = df[df['product_id'] == item]['color_parsed'].mode()[0]
        except IndexError:
            recommendation['Color'][row] = 'No Color'
        try:
            recommendation['Color'][row] = df[df['product_id'] == item]['color_parsed'].mode()[0]
        except IndexError:
            recommendation['Color'][row] = 'No Color'
    return recommendation

In [241]:
def similarity(df):
    users_ids = df.index
    similarity = pd.DataFrame(data = cosine_similarity(df), 
                         index = users_ids, columns = users_ids)
    return similarity

Workflow for a new user

In [242]:
# First page: recommend most popular items

def most_popular(n_items, list_items = [], list_brand = [], list_occasion = [], list_category = [], list_price = []):
    item = filter_item(list_items, list_brand, list_occasion, list_category, list_price)
    popular_item = item.groupby(["product_id", "category_name"])[["has_product"]].sum().reset_index()
    popular_item = popular_item.sort_values(by = "has_product", ascending = False)[:n_items * 5]
    popular_item = random.sample(popular_item['product_id'].tolist(), n_items)
    recommendation = fill_table(item, popular_item, n_items)
    return recommendation

In [243]:
# Second page: select some inputs but do not have a wardrobe
influencers = ['weworewhat', 'chiaraferragni', 'blaireadiebee', 'somethingnavy', 'hannahbronfman',
               'nicolettemason', 'manrepeller', 'jordynwoods', 'seaofshoes', 'ariellecharnas']

def recommendation_influencer_aff(list_influencer, n_item, list_items = [], list_brand = [], list_occasion = [], list_category = [], list_price = []):
    influencer = pd.DataFrame(data = 0, columns = influencers, index = ['new user'])
    for influ in list_influencer:
        influencer[influ] = 1
    similarity_user = similarity(user_influencer.append(pd.DataFrame(influencer)))
    similar_users = similarity_user['new user'].sort_values(ascending = False)[1:]
    if (similar_users[0] == 0):
        recommendation = most_popular(n_item, list_items, list_brand, list_occasion, list_category)
    else:
        similar_users = similar_users[similar_users > 0.5].index
        item = filter_item(list_items, list_brand, list_occasion, list_category, list_price)
        similar_subset = item[item['user_id'].isin(similar_users)]
        similar_subset = similar_subset.groupby(["product_id", "category_name"])[["has_product"]].sum().reset_index()
        similar_subset = similar_subset[similar_subset['has_product'] > 0].sort_values(by = "has_product", ascending = False)
        if len(similar_subset) <= n_item:
            n_item = len(similar_subset)
            similar_subset = similar_subset['product_id']
        else:
            similar_subset = random.sample(similar_subset['product_id'].tolist(), n_item)
        recommendation = fill_table(item, similar_subset, n_item)
    return recommendation


In [244]:
def recommend_new_user(list_influencer, n_recommendations, list_brand, list_occasion, list_category, list_price):
    list_items = []
    if list_influencer == []:
        return most_popular(n_recommendations, list_items, list_brand, list_occasion, list_category, list_price)
    else:
        return recommendation_influencer_aff(list_influencer, n_recommendations, list_items, list_brand, list_occasion, list_category, list_price)

In [292]:
# User selection:
list_influencer = [] # user input
list_occasion = [] # user input
list_category = [] # user input
list_price = ['a']

In [298]:
list_brand = []
n_recommendations = 5
recommend_new_user(list_influencer, n_recommendations, list_brand, list_occasion, list_category, list_price)

,Item Name,Brand,Store,Category,Price,Color,Occasion
0,Godry Frenchterry Run Pant,Old Navy,Old Navy,Pants,26,black,0
1,Aerie Side Snap Culotte,American Eagle Outfitters,American Eagle Outfitters,Pants,26,Slab Gray,0
2,Uo Avery Puddle Pant,Urban Outfitters,Urban Outfitters,Pants,29,No Color,0
3,Nike Baselayer Athletic Capris,No Brand,JC Penney,Pants,30,color: black racer pink size: medium status: i...,0
4,Wit Wisdom Absolution Stretch Twill Skinny Pan...,No Brand,Nordstrom,Pants,42,thyme,0


Workflow for a user that already exists

In [39]:
userBrands = subset_item.groupby(['user_id','brand_id']).count()
userBrands = userBrands.reset_index().pivot(index='user_id', columns='brand_id', values = 'item_name_lower')
userBrands.fillna(0, inplace = True)
brandAffinity = np.ones(len(userBrands.columns)**2).reshape(len(userBrands.columns),len(userBrands.columns))*.05 + np.eye(len(userBrands.columns))
occasions = ['Beach','Work','School','Errands','Dinner','Formal event','Concert',
            'Night life','Cocktail parties','Brunch','Interview','Movies','Workout']

In [307]:
def recommend_user_exists(user_id, n_item, occasion = [], category = [], list_price = []):
    uBrandScore = np.matmul(userBrands.loc[[user_id]],brandAffinity)
    brands = uBrandScore.T.sample(40, replace = True).index.tolist()
    item = subset_item[subset_item['user_id'] == user_id]['product_id'].unique().tolist()
    influencer = user_influencer.filter(like = user_id, axis = 0)
    influencer = [column for column in influencer.columns if influencer.loc[user_id][column] == 1]
    if influencer == []:
        item_to_recommend = most_popular(n_item, item, brands, occasion, category, list_price)
    else:
        item_to_recommend = recommendation_influencer_aff(influencer, n_item, item, brands, occasion, category, list_price)
    return item_to_recommend

In [312]:
user_id = subset_item['user_id'].sample(1).values[0] # user input
list_occasion = ['Work'] # user input
list_category = [] # user input
list_price = ['b']

In [313]:
recommend_user_exists(user_id, n_recommendations, list_occasion, list_category, list_price)

,Item Name,Brand,Store,Category,Price,Color,Occasion
0,Strappy Long Line Top,Victorias Secret,Victoria's Secret,Blouses,48,black (093),0
1,Vs Siren Cargo Ankle Jean,Victorias Secret,Victoria's Secret,Denim,65,camo (26d),0
2,Dot Print Wrap Skirt Dress,Muche Et Muchette,HauteLook,Casual,44,No Color,0
3,Pointed Toe Platform Pump,Victorias Secret,Victoria's Secret,Heels,69,black (093),0
4,Colorblock Crewneck Sweater,Victorias Secret,Victoria's Secret,Sweaters,44,snow heather (eb8),0


In [228]:
mean_price = subset_item.groupby(['brand_id', 'category_name'])[['paid_price']].mean().reset_index()
mean_price.head()

,brand_id,category_name,paid_price
0,14thunion,Sandals,29.980000
1,14thunion,T-shirts,12.970000
2,1state,Miscellaneous,20.700000
3,7forallmankind,Bottoms,92.390000
4,7forallmankind,Denim,96.666667


In [207]:
mean_price_cat = subset_item.groupby(['category_name'])[['paid_price']].mean().reset_index()
mean_price_cat.head()

,category_name,paid_price
0,Accessories,14.994286
1,Activewear,51.542919
2,Backpacks,86.650000
3,Bags,207.230769
4,Beachwear,23.660000


In [202]:
ids = subset_item.loc[(subset_item['paid_price'].isnull()) & (subset_item['category_name'].isnull()) & (subset_item['brand_id'].isnull())].index
for i in ids:
    subset_item.loc[i, "paid_price"] = 0

In [223]:
ids = subset_item.loc[(subset_item['paid_price'].isnull()) & (subset_item['brand_id'].isnull())].index
for i in ids:
    category = subset_item.loc[i, 'category_name']
    price = mean_price_cat[(mean_price_cat['category_name'] == category)]['paid_price']
    subset_item.loc[i, "paid_price"] = int(price)


In [236]:
ids = subset_item.loc[(subset_item['paid_price'].isnull())].index
for i in ids:
    brand = subset_item.loc[i, 'brand_id']
    category = subset_item.loc[i, 'category_name']
    price = mean_price[(mean_price['category_name'] == category) & (mean_price['brand_id'] == brand)]['paid_price']
    if np.isnan(price).all() == True:
        subset_item.loc[i, "paid_price"] = 0
    else:
        subset_item.loc[i, "paid_price"] = int(price)

In [237]:
subset_item.loc[(subset_item['paid_price'].isnull())]

,user_id,product_id,brand_id,parsed_brand_name,store_id,parsed_store_name,is_deleted,item_name_lower,item_name,category_id,category_name,occasion,paid_price,color_parsed,wishlist,has_product


In [ ]:
# Brings 5 similar items based on cosine similarity
def get_similar_items(x):
   return '\n'.join(item_subset['item_new'].iloc[x.argsort()[-10:-1]])

# Need everything as string
item_subset = subset_item.astype(str)

# Prepping data for analysis
item_subset['item_new'] = (item_subset['brand_id'] + ', ' +
                      item_subset['item_name_lower'])

# Func to do analysis
Text = item_subset['item_new'].tolist()
tfidf = text.TfidfVectorizer(input=Text, ngram_range=(1, 2), stop_words='english')
matrix = tfidf.fit_transform(Text)
print(matrix.shape)

# Get Similarity Scores using cosine similarity
sim_unigram = linear_kernel(matrix, matrix)

# appends the new items in the new column {‘similar items’]
item_subset['similar_items']=[get_similar_items(x) for x in sim_unigram]


In [ ]:
product_id = "58d5c54d5e4fefcabb201c3c"
#item_split = item_subset[item_subset['product_id'] == product_id]['similar_items'].split('\n')
#item_split
item_subset['similar_items'][0]
item_split = item_subset[item_subset['product_id'] == product_id]['similar_items']
item_split = item_split.split('\n')
item_split

In [ ]:
def recommend_similar_item(products_id, n_rec)
    item_split = item_subset['similar_items'][0].split('\n')
    item_subset[item_subset['item_new'].str.match(item_split[0])]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

# Need everything as string
item_subset = subset_item.astype(str)

# Prepping data for analysis
item_subset['item_new'] = (item_subset['brand_id'] + ', ' +
                      item_subset['item_name_lower'])

# Func to do analysis
Text = item_subset['item_new'].tolist()
tfidf = text.TfidfVectorizer(input=Text, ngram_range=(1, 2), stop_words='english')
matrix = tfidf.fit_transform(Text)
print(matrix.shape)

# Get Similarity Scores using cosine similarity
sim_unigram = linear_kernel(matrix, matrix)

# Brings 5 similar items based on cosine similarity
def get_similar_items(x):
   return '\n'.join(item_subset['item_new'].iloc[x.argsort()[-10:-1]])

# appends the new items in the new column {‘similar items’]
item_subset['similar_items']=[get_similar_items(x) for x in sim_unigram]

item_split = item_subset['similar_items'][0].split('\n')

item_subset[item_subset['item_new'].str.match(item_split[0])]
#item_split
# shows the new df
#item_subset

In [ ]:
item_subset

# Stella's code

In [ ]:
def filter_item(list_brand = None, list_occasion = None, list_category = None):
    df = subset_item[subset_item['category_name'] != 'Beauty']
    if list_brand != None:
        df = df[df['brand_id'].isin(list_brand)]
    if list_occasion != None:
        list_occasion = "|".join(list_occasion)
        df = df[df['occasion'].str.contains(list_occasion, na = False)]
    if list_category != None:
        df = df[df['category_name'].isin(list_category)]
    return df

In [ ]:
def fill_table(df, product_id, n_items):
    recommendation = pd.DataFrame(data = 0, index = range(n_items), 
                                  columns = ['Ranking', 'Item ID', 'Item Name', 'Store', 'Category', 'Price', 'Color'])
    for row, item in enumerate (product_id):
        recommendation['Ranking'][row] = row + 1
        recommendation['Item ID'][row] = item
        recommendation['Item Name'][row] = df[df['product_id'] == item]['item_name_lower'].mode()[0]
        recommendation['Store'][row] = df[df['product_id'] == item]['store_id'].mode()[0]
        try:
            recommendation['Category'][row] = df[df['product_id'] == item]['category_name'].mode()[0]
        except IndexError:
            recommendation['Category'][row] = 'no category'
        recommendation['Price'][row] = df[df['product_id'] == item]['paid_price'].mean()
        try:
            recommendation['Color'][row] = df[df['product_id'] == item]['color_parsed'].mode()[0]
        except IndexError:
            recommendation['Color'][row] = 'no color'
    return recommendation

In [ ]:
def most_popular(n_items, list_brand = None, list_occasion = None, list_category = None):
    item = filter_item(list_brand, list_occasion, list_category)
    popular_item = item.groupby(["product_id", "category_name"])[["has_product"]].sum().reset_index()
    popular_item = popular_item.sort_values(by = "has_product", ascending = False)[:n_items * 3]
    recommendation = fill_table(item, popular_item['product_id'][:n_items], n_items)
    return recommendation

In [ ]:
most_popular(5)

In [ ]:
def similarity(df):
    users_ids = df.index
    similarity = pd.DataFrame(data = cosine_similarity(df), 
                         index = users_ids, columns = users_ids)
    return similarity

In [ ]:
influencers = ['weworewhat', 'chiaraferragni', 'blaireadiebee', 'somethingnavy', 'hannahbronfman',
               'nicolettemason', 'manrepeller', 'jordynwoods', 'seaofshoes', 'ariellecharnas']

def recommendation_influencer_aff(list_influencer, n_item, list_brand = None, list_occasion = None, list_category = None):
    influencer = pd.DataFrame(data = [list_influencer], columns = influencers)
    similarity_user = similarity(user_influencer.append(pd.DataFrame(influencer)))
    similar_users = similarity_user[0].sort_values(ascending = False)[1:]
    similar_users = similar_users[similar_users > 0.5].index
    item = filter_item(list_brand, list_occasion, list_category)
    similar_subset = item[item['user_id'].isin(similar_users)]
    similar_subset = similar_subset.groupby(["product_id", "category_name"])[["has_product"]].sum().reset_index()
    similar_subset = similar_subset[similar_subset['has_product'] > 0].sort_values(by = "has_product", ascending = False)
    recommendation = fill_table(item, similar_subset['product_id'][:n_item], n_item)
    return recommendation

In [ ]:
list_influencer = [0,1,0,0,0,0,0,0,0,0]
recommendation_influencer_aff(list_influencer, 5)

# Greg's Code

In [ ]:
#Finery Recommender Function
import numpy as np
import pandas as pd
from random import choice, sample
import random

items = pd.read_csv('../data/items_fixed.csv')
userBrands = items.groupby(['user_id','brand']).count()
userBrands = userBrands.reset_index().pivot(index='user_id', columns='brand', values = 'item')
userBrands.fillna(0, inplace = True)
brandAffinity = np.ones(len(userBrands.columns)**2).reshape(len(userBrands.columns),len(userBrands.columns))*.05 + np.eye(len(userBrands.columns))
occasions = ['Beach','Work','School','Errands','Dinner','Formal event','Concert',
            'Night life','Cocktail parties','Brunch','Interview','Movies','Workout']


In [ ]:
def recommend(user_id = items['user_id'].sample(1).values[0], occasion = random.choice(occasions),
              category = items['category_name'].sample(1).values[0]):
    uBrandScore = np.matmul(userBrands.loc[[user_id]],brandAffinity)
    brands = uBrandScore.T.sample(20, replace = True).index
    try:
        item_to_recommend = items.loc[(items['brand'].isin(brands)) & 
                            (items['occasions'].str.contains(occasion)) & 
                            (items['category_name'] == category)].sample(1)[['item','brand','occasions']]
        return item_to_recommend
    except:
        return recommend(user_id, occasion, category)

In [ ]:
recommend(user_id = 'uDlpcEvrISMoLvJKh1517962134688', occasion = "Work", category = "Jackets")